# Query Expansion

0) Just some imports

In [68]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [94]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeIDFs(self, documents):
        self.idfs = []
        
        #moj kod
        base = len(documents)
        for term in self.terms:
            x = 0.0
            for document in documents:
                if term in document.bow:
                    x += 1.0
            if x==0:
                x = 1
            myLog = math.log(base/x, base)
            self.idfs.append(myLog)
    
    ### TODO (DO NOT FINISH THIS METHOD YET. YOU WILL BE ASKED FOR IT LATER) 
    def computeCorM(self, documents):
        self.corM = np.zeros((len(self.terms), len(documents)))
        t = []
        
        for i, term in enumerate(self.terms):
            for j, document in enumerate(documents):
                self.corM[i][j] = document.bow[i]
            t.append(sum(self.corM[i]))
            
        for i, term in enumerate(self.terms):
            for j, document in enumerate(documents):
                self.corM[i][j] = self.corM[i][j] / t[i]
        transpose = np.transpose(self.corM)
        
        self.corM = np.dot(self.corM, transpose)
        
        for i in range(len(self.corM)):
            self.corM[i][i] = -1
        
                        
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [95]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [96]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [97]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    ### TODO - complete this method; it should compute a BOW representation
    def computeBOW_Representation(self):
        self.bow = []
        
        #ilosc wystapien danego termu - przejsc po termach i spr ile razy wystepuje w dokumencie? długie, ale powinno działać?
        if self.dictionary.terms != None:
            for index, term in enumerate(self.dictionary.terms):
                self.bow.append(0)      #jak nie zadziala to tu append zrobic
                for elem in self.tokens:
                    if (elem == term):
                        self.bow[index] += 1
       
    
    
    ### TODO - complete this method; it should compute a TF representation
    def computeTF_Representation(self):
        self.tf = []
        
        #wystarczy max z termow i normalizacja przez to?
        myMax = 1
        for elem in self.bow:
            if (elem >= myMax):
                myMax = elem
        
        for elem in self.bow:
            self.tf.append(elem / myMax)
        
        
        
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        self.tf_idf = []
        
        if dictionary.idfs != None:
            for index in range(len(self.bow)):
                self.tf_idf.append(self.bow[index] * dictionary.idfs[index])
            
        #tu troche nie wiem, najpierw spr ten warunek, potem wyliczyc log i na koniec w petli przemnozyc???
        
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]


1.4) Compute IDFs here

In [98]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms))]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print()
print(res[:5])

[['yahoo', 1.0], ['york', 1.0], ['you', 1.0], ['young', 1.0], ['zdm', 1.0]]

[['aaai', 1.0], ['about', 1.0], ['academic', 1.0], ['access', 1.0], ['acquired', 1.0]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [99]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(documents[0].bow)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [100]:
### TODO 
def getSimilarity(v1, v2):      #wzor suma(a*b) / sqrt (suma (a)^2   * suma (b)^2) 
    mySum = 0
    sumA = 0
    sumB = 0
    for elemA, elemB in zip(v1,v2):
        mySum += elemA * elemB
        sumA += elemA * elemA
        sumB += elemB * elemB
    
    
    return mySum / (math.sqrt (sumA * sumB))

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [128]:
query = "machine learning"
#query = "academic research"
#query = "international conference"
#query = "international conference washington"

In [129]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.8908708063747479 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.8728715609439696 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 3 WITH SIMILARITY = 0.8703882797784892 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 4 WITH SIMILARITY = 0.8366600265340756 | DOC ID = 28
Machine Learning
 6.858/18.428: Machine Learning. Available Lecture Notes. ... Defining models for
 machine learning. Learning conjunctions in the mistake-bounded model. ... 


RANK = 5 WITH SIMILARITY = 0.8249579113843055 | DOC ID = 61
CS 661 - Machi

# 2) Query expansion

## 2.1) Correlation matrix

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [103]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(dictionary.corM)

[[-1.    0.    0.   ...  0.    0.    0.  ]
 [ 0.   -1.    0.   ...  0.05  0.    0.  ]
 [ 0.    0.   -1.   ...  0.    0.    0.  ]
 ...
 [ 0.    0.05  0.   ... -1.    0.    0.  ]
 [ 0.    0.    0.   ...  0.   -1.    0.  ]
 [ 0.    0.    0.   ...  0.    0.   -1.  ]]


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [126]:
#query = "machine"
#query = "algorithm"
#query = "learning"
#query = "conference"
#query = "research"
query = "concept"

def suggestKeywords(query, dictionary):
    ### TODO
    #print("Suggestions")
    #pass
    lookFor = -1
    for index, term in enumerate(dictionary.terms):
        if query == term:
            lookFor = index
            break
    maxValue = max(dictionary.corM[index])
    for i, elem in enumerate(dictionary.corM[index]):
        if elem == maxValue:
            return dictionary.terms[i]
        
suggestKeywords(query, dictionary)

129


'deals'

# 2.2) Rocchio algorithm

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [142]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.9,
                          beta = 0.6,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    
    dj = 0     #bedzie juz przechowywac cala sume, a nie poszczegolne dj
    ndj = 0
    
    #TU NIE WIEM CO TO WŁAŚCIWIE JEST dj we Wzorze, czy jest ok?
    for d in documents:
        if d.doc_id in rel_docs:
            dj += getSimilarity(d.tf_idf, qd.tf_idf)
        if d.doc_id in nrel_docs:
            ndj += getSimilarity(d.tf_idf, qd.tf_idf)
            
    
    for i, elem in enumerate(qd.tf_idf):
        qd.tf_idf[i] = alpha * elem + (beta / len(rel_docs) * dj) - (gamma / len(nrel_docs) * ndj)
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

1.4758181820886294
1.7612590861532371
RANK = 1 WITH SIMILARITY = 0.3024566686388972 | DOC ID = 52
Machine Learning at UC Santa Cruz
 Machine Learning Home Page. ... The Machine Learning group at UCSC is dedicated to
 the discovery and analysis of algorithms for different learning problems. ... 
 Description: Research on decision theory, neural networks, computational biology, computational geometry, theoretical...


RANK = 2 WITH SIMILARITY = 0.29933636137308217 | DOC ID = 27
Machine Learning FAQ
 Machine Learning FAQ. Machine Learning Frequently Asked Questions. Click Here
 to access our page. You are using a Browser that does not support Frames. ... 
 Description: Questions and answers about Machine Learning. Anyone can post his/her own questions and answers.


RANK = 3 WITH SIMILARITY = 0.2965913359686126 | DOC ID = 0
David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine

# 2.3) WordNet

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [148]:
import nltk

nltk.download()
from nltk.corpus import wordnet as wn

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [149]:
wn.synsets('machine')

[Synset('machine.n.01'),
 Synset('machine.n.02'),
 Synset('machine.n.03'),
 Synset('machine.n.04'),
 Synset('machine.n.05'),
 Synset('car.n.01'),
 Synset('machine.v.01'),
 Synset('machine.v.02')]

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [152]:
#TODO
for elem in wn.synsets('machine'):
    print(elem.definition())

any mechanical or electrical device that transmits or modifies energy to perform or assist in the performance of human tasks
an efficient person
an intricate organization that accomplishes its goals efficiently
a device for overcoming resistance at one point by applying force at some other point
a group that controls the activities of a political party
a motor vehicle with four wheels; usually propelled by an internal combustion engine
turn, shape, mold, or otherwise finish by machinery
make by machinery


2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [154]:
#TODO
for elem in wn.synsets('machine'):
    print(elem.hypernyms())

[Synset('device.n.01')]
[Synset('person.n.01')]
[Synset('organization.n.01')]
[Synset('mechanical_device.n.01')]
[Synset('organization.n.01')]
[Synset('motor_vehicle.n.01')]
[Synset('shape.v.02')]
[Synset('produce.v.02')]


See: http://www.nltk.org/howto/wordnet.html
for more examples